In [77]:
import pandas as pd
import lightgbm as lgb
import time
path='/home/student303/zz/almama/'

#Read Data
data = pd.read_hdf(path+ 'somefeature/base_feature_all_0.h5', key = 'a')
print('Read Data Finish.')

Read Data Finish.


In [46]:
#Define featurelist
droplist=[
          'item_category_list',
          'item_property_list',
          'predict_category_property',
          
          'shop_review_positive_rate',
          'shop_score_service',
          'shop_score_delivery',
          'shop_score_description'
         ]
# featurelist=[i for i in data.columns if i not in droplist]

In [47]:
data_cal = data.drop(droplist,axis=1)

In [48]:
for i in data_cal.columns:
    merger = data.groupby(i).size().reset_index().rename(columns={0:i +'_cNt'})
    data = pd.merge(data, merger, on=i, how='left')

## next is two columns

In [78]:
sUm = 0
for i in range(1,len(data_cal.columns)):
    for j in range(i+1,len(data_cal.columns)):
        #print(data_cal.columns[i]+'_'+data_cal.columns[j] +'_cNt')
        merger = data.groupby([data_cal.columns[i],data_cal.columns[j]]).size().reset_index().rename(columns={0:data_cal.columns[i]+'_'+data_cal.columns[j] +'_cNt'})
        data = pd.merge(data, merger, on=[data_cal.columns[i],data_cal.columns[j]], how='left')
        sUm = sUm +1
        print('we done ' +data_cal.columns[i]+'_'+data_cal.columns[j]+'_cNt',end=' ')
        print(sUm)

we done item_id_item_brand_id_cNt 1
we done item_id_item_city_id_cNt 2
we done item_id_item_price_level_cNt 3
we done item_id_item_sales_level_cNt 4
we done item_id_item_collected_level_cNt 5
we done item_id_item_pv_level_cNt 6
we done item_id_user_id_cNt 7
we done item_id_user_gender_id_cNt 8
we done item_id_user_age_level_cNt 9
we done item_id_user_occupation_id_cNt 10
we done item_id_user_star_level_cNt 11
we done item_id_context_id_cNt 12
we done item_id_context_timestamp_cNt 13
we done item_id_context_page_id_cNt 14
we done item_id_shop_id_cNt 15
we done item_id_shop_review_num_level_cNt 16
we done item_id_shop_star_level_cNt 17
we done item_id_is_trade_cNt 18
we done item_id_item_category_hash_cNt 19
we done item_id_day_cNt 20
we done item_id_hour_cNt 21
we done item_id_min_cNt 22
we done item_id_second_cNt 23
we done item_brand_id_item_city_id_cNt 24
we done item_brand_id_item_price_level_cNt 25
we done item_brand_id_item_sales_level_cNt 26
we done item_brand_id_item_collected_l

In [79]:
#Define featurelist
droplist=['instance_id',
          'item_category_list',
          'item_property_list',
          'user_id', 
          'context_id',
          'context_timestamp',
          'predict_category_property',
          'is_trade',
          'day', 'min', 'second',
         ]
featurelist=[i for i in data.columns if i not in droplist]

In [83]:
%%time
dev=1
print('LGBM Training, Feature dim : ',len(featurelist))
params = {
    'task': 'train',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'max_depth':10,
    #'num_leaves': 6,
    #'min_data_in_leaf':1,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 1.0,
    'verbose': 0,
}
if dev:
    d_train=lgb.Dataset(data.loc[data.day < 24][featurelist], label=data.loc[data.day < 24]['is_trade'])
    d_valid=lgb.Dataset(data.loc[data.day == 24][featurelist], label=data.loc[data.day == 24]['is_trade'])
    model = lgb.train(params, train_set=d_train, num_boost_round=600, valid_sets=[d_train,d_valid], \
                  early_stopping_rounds=50, verbose_eval=10)
    offlinelogloss=model.best_score['valid_1']['binary_logloss']
    bst_iteration=model.best_iteration
else:
    d_train=lgb.Dataset(data.loc[data.day <= 24][featurelist], label=data.loc[data.day <= 24]['is_trade'])    
    model = lgb.train(params, train_set=d_train, num_boost_round=bst_iteration, valid_sets=[d_train], \
                   verbose_eval=50)
print('LGBM Training Done')

LGBM Training, Feature dim :  297
Train until valid scores didn't improve in 50 rounds.
[10]	training's binary_logloss: 0.19756	valid_1's binary_logloss: 0.19756
[20]	training's binary_logloss: 0.0677582	valid_1's binary_logloss: 0.0677582
[30]	training's binary_logloss: 0.0243418	valid_1's binary_logloss: 0.0243418
[40]	training's binary_logloss: 0.00887907	valid_1's binary_logloss: 0.00887907
[50]	training's binary_logloss: 0.00325634	valid_1's binary_logloss: 0.00325634
[60]	training's binary_logloss: 0.00119658	valid_1's binary_logloss: 0.00119658
[70]	training's binary_logloss: 0.000440014	valid_1's binary_logloss: 0.000440014
[80]	training's binary_logloss: 0.000161847	valid_1's binary_logloss: 0.000161847
[90]	training's binary_logloss: 5.9537e-05	valid_1's binary_logloss: 5.9537e-05
[100]	training's binary_logloss: 2.1902e-05	valid_1's binary_logloss: 2.1902e-05
[110]	training's binary_logloss: 8.05722e-06	valid_1's binary_logloss: 8.05722e-06
[120]	training's binary_logloss: 2